In [4]:
# import pandas as pd
# from flask_sqlalchemy import SQLAlchemy
# from sqlalchemy import create_engine
# import regex as re
# import time
# from datetime import datetime
# import os

# # prod
# user = 'read_only'
# pwd = 'WHC63fKyjudfSz2U'
# host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# db_name = 'CAPC_APIGATEWAY'
# env='production'

# db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
# db_connection = create_engine(db_connection_str)

In [6]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os
import pymysql



# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'


# # stag
# user = 'read_only'
# pwd = '7TYtBma28775c4Ua'
# host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# db_name = 'CAPC_APIGATEWAY'
# env='staging'


db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

ModuleNotFoundError: No module named 'pymysql'

In [2]:
# acc = ['V00001027861', 'V00001028015', 'V00001027870', 'V00001028313', 'V00001028322', 'V00001028498', 'V00001028630', 'V00001028745', 'V00001029577', 'V00001029332', 'V00001029341', 'V00001029954', 'V00001029984', 'V00001029216', 'V00001029235', 'V00001029244', 'V00001029223', 'V00001030368', 'V00001030938', 'V00001030946', 'V00001031087', 'V00001031248', 'V00001031227', 'V00001031254', 'V00001031443', 'V00001031449', 'V00001031428', 'V00001031438', 'V00001031345', 'V00001031674', 'V00001031784', 'V00001031826', 'V00001031896', 'V00001032131', 'V00001032451', 'V00001032326', 'V00001032470', 'V00001032582', 'V00001032633', 'V00001032639', 'V00001032622', 'V00001032653', 'V00001032853', 'V00001032876', 'V00001032874', 'V00001033054', 'V00001033055', 'V00001033059', 'V00001033281', 'V00001033285', 'V00001033299', 'V00001033302', 'V00001033307', 'V00001033301', 'V00001033392', 'V00001033415', 'V00001033421', 'V00001033578', 'V00001033588', 'V00001034616', 'V00001034585', 'V00001034881', 'V00001035079', 'V00001035607', 'V00001036030', 'V00001036089', 'V00001036344', 'V00001036346', 'V00001036366', 'V00001036340', 'V00001036379', 'V00001036461', 'V00001036504', 'V00001036528', 'V00001036467', 'V00001036508', 'V00001036701', 'V00001036709', 'V00001036669', 'V00001036678', 'V00001036265', 'V00001036921', 'V00001036949', 'V00001036976', 'V00001037431', 'V00001036229', 'V00001036545', 'V00001037599', 'V00001037736', 'V00001037690', 'V00001037851', 'V00001037869', 'V00001038087', 'V00001038545', 'V00001038817', 'V00001038784', 'V00001038776', 'V00001038908', 'V00001038874', 'V00001038905', 'V00001038926', 'V00001038927', 'V00001038922', 'V00001038906', 'V00001038749', 'V00001038386', 'V00001039459', 'V00001039676', 'V00001039942', 'V00001040962', 'V00001040975', 'V00001041224', 'V00001041295', 'V00001042758', 'V00001042779', 'V00001042763', 'V00001042774', 'V00001042921', 'V00001042931', 'V00001042942', 'V00001042941', 'V00001042944', 'V00001042965', 'V00001043313', 'V00001043475', 'V00001043992']

In [18]:
query = '''SELECT 
    CONVERT_TZ(em.service_date, 'UTC', 'US/Central') AS service_date,
    CONVERT_TZ(dpd.created_date, 'UTC', 'US/Central') AS processing_date,
    GROUP_CONCAT(DISTINCT eandm.code) AS final,
    GROUP_CONCAT(DISTINCT dc.code) AS suggested,
    GROUP_CONCAT(DISTINCT dc.time_spent_in_mins) AS time_spent,
    am.account_number AS account_number,
    facility_id, client_id
FROM
    account_mst am
        INNER JOIN
    document_mst dm ON dm.account_id = am.id
        AND dm.is_physician_document = 1
        AND dm.is_active = 1
        INNER JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    document_code dc ON dc.document_id = dm.id
        AND dc.is_active = 1
        AND dc.code_category = 'EANDM'
        AND dc.calculation_mode = 'TIME'
        INNER JOIN
    encounter_mst em ON em.account_id = dm.account_id
        AND em.service_date = dm.service_date
        AND em.physician_id = dm.physician_id
        AND em.is_active = 1
        INNER JOIN
    encounter_status_map esm ON esm.encounter_id = em.id
        AND esm.is_active = 1
        AND esm.encounter_status_id IN (9 , 17)
        INNER JOIN
    eandm_code eandm ON eandm.encounter_id = em.id
        AND eandm.process_id = esm.process_id
        AND eandm.user_id = esm.user_id
        AND eandm.status = 'ACCEPTED'
        AND eandm.is_active = 1
WHERE
    am.account_number in ({})
    
    AND dpd.id = (SELECT 
            id
        FROM
            document_processing_detail
        WHERE
            document_id = dm.id
        ORDER BY created_date DESC
        LIMIT 1)
        AND esm.id = (SELECT 
            id
        FROM
            encounter_status_map
        WHERE
            encounter_id = em.id
                AND encounter_status_id IN (9 , 17)
        ORDER BY last_coding_date DESC
        LIMIT 1)
        AND am.is_active = 1

        AND CONVERT_TZ(em.service_date,
            'UTC',
            'US/Central') > '2023-03-01'
        -- AND dc.code LIKE '9929%%'
        AND eandm.code LIKE '99406'
        AND am.facility_id = '221'
GROUP BY am.account_number , em.physician_id , em.service_date , dm.id
ORDER BY am.facility_id DESC;'''.format(','.join(['"'+str(x)+'"' for x in acc]))

        # AND CONVERT_TZ(esm.last_coding_date,
        #     'UTC',
        #     'US/Central') >= '2023-03-15'
        # AND CONVERT_TZ(esm.last_coding_date,
        #     'UTC',
        #     'US/Central') < '2023-04-24'

In [19]:
# query = '''
# SELECT 
#     CONVERT_TZ(em.service_date, 'UTC', 'US/Central') AS service_date,
#     GROUP_CONCAT(DISTINCT eandm.code) AS final,
#     GROUP_CONCAT(DISTINCT dc.code) AS suggested,
#     am.account_number AS account_number,
#     facility_id,
#     client_id
# FROM
#     account_mst am
#         INNER JOIN
#     document_mst dm ON dm.account_id = am.id
#         AND dm.is_physician_document = 1
#         AND dm.is_active = 1
#         INNER JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     document_code dc ON dc.document_id = dm.id
#         AND dc.is_active = 1
#         AND dc.code_category = 'EANDM'
#         INNER JOIN
#     encounter_mst em ON em.account_id = dm.account_id
#         AND em.service_date = dm.service_date
#         AND em.physician_id = dm.physician_id
#         AND em.is_active = 1
#         INNER JOIN
#     encounter_status_map esm ON esm.encounter_id = em.id
#         AND esm.is_active = 1
#         AND esm.encounter_status_id IN (9 , 17)
#         INNER JOIN
#     eandm_code eandm ON eandm.encounter_id = em.id
#         AND eandm.process_id = esm.process_id
#         AND eandm.user_id = esm.user_id
#         AND eandm.is_active = 1
# WHERE
#     dpd.id = (SELECT 
#             id
#         FROM
#             document_processing_detail
#         WHERE
#             document_id = dm.id
#         ORDER BY created_date DESC
#         LIMIT 1)
#         AND am.facility_id = '221'
#         AND esm.id = (SELECT 
#             id
#         FROM
#             encounter_status_map
#         WHERE
#             encounter_id = em.id
#                 AND encounter_status_id IN (9 , 17)
#         ORDER BY last_coding_date DESC
#         LIMIT 1)
#         AND am.is_active = 1
#         AND CONVERT_TZ(esm.last_coding_date,
#             'UTC',
#             'US/Central') >= '2023-05-01'
# GROUP BY am.account_number , em.physician_id , em.service_date , dm.id
# '''

In [20]:
df = pd.read_sql(query, con=db_connection)
# print(len(df_all))

In [21]:
df.facility_id.unique()

array([221])

In [23]:
df.to_excel('/home/trishalaswain/ClientData/PH/CHINO_MONTH_WISE/CHINO_ALL_OP/chino_time_backTracking.xlsx')

In [29]:
df.to_excel('/home/trishalaswain/Downloads/Script_Testing/3035_may_till9_accounts_sheet.xlsx')

In [32]:
df_92 = df[df.final.str.contains('99291,99292')]
#  & (df.facility_id == 38)

In [33]:
df_91 = df[df.final == '99291']

In [35]:
df_91.sort_values('time_spent', ascending=False)

,service_date,final,suggested,time_spent,account_number,facility_id,client_id
293,2023-04-09 02:00:00,99291,"99291,99292",95,V00001041295,221,13
370,2023-04-20 02:00:00,99291,"99291,99292",95,V00001042931,221,13
479,2023-05-04 02:00:00,99291,99291,95,V00001044980,221,13
209,2023-03-30 02:00:00,99291,"99291,99292",95,V00001039942,221,13
475,2023-05-04 02:00:00,99291,99291,95,V00001044959,221,13
...,...,...,...,...,...,...,...
149,2023-03-21 02:00:00,99291,"99291,99292",120,V00001038784,221,13
507,2023-05-10 02:00:00,99291,99291,100,V00001045743,221,13
134,2023-03-20 02:00:00,99291,"99291,99292",100,V00001038608,221,13
514,2023-05-12 02:00:00,99291,99291,100,V00001045937,221,13


In [38]:
pd.DataFrame(df.facility_id.value_counts()).head(15)

,facility_id
221,554


In [107]:
df[df.facility_id==40].sort_values('time_spent')

,service_date,final,suggested,time_spent,account_number,facility_id,client_id
31830,2023-03-02 00:09:06,99291,99291,30,13015553,40,13
31926,2023-03-02 15:23:54,99291,99291,30,13165820,40,13
32894,2023-04-05 19:47:49,99291,99291,30,13195347,40,13
32227,2023-03-14 00:36:15,99291,99291,30,13175660,40,13
32234,2023-03-14 12:23:21,99291,99291,30,13176049,40,13
...,...,...,...,...,...,...,...
31977,2023-03-04 18:33:24,99291,"99291,99292",540,13167590,40,13
32564,2023-03-25 22:27:39,99291,"99291,99292",540,13185739,40,13
32131,2023-03-10 15:31:35,"99291,99292","99291,99292",600,13173111,40,13
31992,2023-03-05 06:42:05,99291,"99291,99292",660,13167803,40,13


In [108]:
new_df = df[df.facility_id==40].sort_values(['facility_id', 'time_spent'])

In [39]:
new_df.time_spent.describe()

count     554
unique     31
top        35
freq      113
Name: time_spent, dtype: object

In [110]:
df_with_time70_or_above = new_df[new_df.time_spent>70].copy()

In [111]:
df2 = df_with_time70_or_above.sort_values('time_spent')

In [112]:
df2

,service_date,final,suggested,time_spent,account_number,facility_id,client_id
31989,2023-03-05 02:41:51,99291,99291,74,13167761,40,13
32971,2023-04-08 10:01:22,99291,"99291,99292",75,13197804,40,13
32908,2023-04-06 09:34:00,99291,"99291,99292",75,13195613,40,13
32741,2023-03-31 18:39:52,99291,"99291,99292",75,13191086,40,13
32663,2023-03-29 04:12:06,99291,"99291,99292",75,13188366,40,13
...,...,...,...,...,...,...,...
31977,2023-03-04 18:33:24,99291,"99291,99292",540,13167590,40,13
32564,2023-03-25 22:27:39,99291,"99291,99292",540,13185739,40,13
32131,2023-03-10 15:31:35,"99291,99292","99291,99292",600,13173111,40,13
31992,2023-03-05 06:42:05,99291,"99291,99292",660,13167803,40,13


In [119]:
df2[df2.final.str.contains('99292')]

,service_date,final,suggested,time_spent,account_number,facility_id,client_id
32137,2023-03-10 20:07:08,"99291,99292","99291,99292",75,13173371,40,13
31963,2023-03-03 18:01:04,"99291,99292","99291,99292",75,13167059,40,13
31960,2023-03-03 16:39:01,"99291,99292","99291,99292",75,13166985,40,13
32782,2023-04-02 03:10:29,"99291,99292","99291,99292",78,13191784,40,13
31959,2023-03-03 15:40:02,"99291,99292","99291,99292",80,13166913,40,13
32616,2023-03-27 11:09:49,"99291,99292","99291,99292",120,13186522,40,13
32615,2023-03-27 11:09:49,"99291,99292","99291,99292",120,13186522,40,13
33031,2023-04-10 20:37:59,"99291,99292","99291,99292",120,13199550,40,13
32327,2023-03-18 02:14:34,"99291,99292","99291,99292",120,13179803,40,13
32080,2023-03-08 16:46:42,"99291,99292","99291,99292",120,13171184,40,13


In [52]:
df.time_spent = df.time_spent.astype('int64')

In [63]:
df = df[df.service_date > '2023-04-15']

In [64]:
df[(~df.final.str.contains('99292')) & (df.time_spent<105)].sort_values('time_spent')

TypeError: '<' not supported between instances of 'str' and 'int'

In [114]:
df_with_time70_or_above['bins'] = pd.cut(df_with_time70_or_above.time_spent, [ 30, 40, 50, 60, 71.,   74.,  75.,78.,79., 80., 81., 82., 83., 84.,85.,  90., 100, 102, 103, 104,  105, 106, 110, 120., 700.])

In [115]:
for i in df_with_time70_or_above.bins.unique():
    temp = df_with_time70_or_above[df_with_time70_or_above.bins==i]
    print(i)
    print(temp['final'].value_counts())
    print(temp['suggested'].value_counts())
    print((temp['final']==temp['suggested']).sum())
    print((temp['final']!=temp['suggested']).sum())
    print()

(71.0, 74.0]
99291    1
Name: final, dtype: int64
99291    1
Name: suggested, dtype: int64
1
0

(74.0, 75.0]
99291          12
99291,99292     3
Name: final, dtype: int64
99291,99292    15
Name: suggested, dtype: int64
3
12

(75.0, 78.0]
99291          12
99291,99292     1
Name: final, dtype: int64
99291,99292    13
Name: suggested, dtype: int64
1
12

(79.0, 80.0]
99291          3
99291,99292    1
Name: final, dtype: int64
99291,99292    4
Name: suggested, dtype: int64
1
3

(85.0, 90.0]
99291    6
Name: final, dtype: int64
99291,99292    6
Name: suggested, dtype: int64
0
6

(90.0, 100.0]
99291    2
Name: final, dtype: int64
99291,99292    2
Name: suggested, dtype: int64
0
2

(110.0, 120.0]
99291          9
99291,99292    6
Name: final, dtype: int64
99291,99292    15
Name: suggested, dtype: int64
6
9

(120.0, 700.0]
99291          66
99291,99292     3
Name: final, dtype: int64
99291,99292    69
Name: suggested, dtype: int64
3
66

